In [1]:
import porespy as ps
import numpy as np
import particle_generator as pg

import supervisely_lib as sly
import os
import skimage as ski
from skimage import color
from skimage import exposure
from skimage import transform

from matplotlib import pyplot as plt

from supervisely_lib.geometry.bitmap import Bitmap
from supervisely_lib.annotation.annotation import Annotation
from supervisely_lib.annotation.label import Label
from supervisely_lib.annotation.obj_class import ObjClass
from supervisely_lib.project.project_meta import ProjectMeta
from supervisely_lib.geometry.point_location import PointLocation

In [2]:
# Change this field to the name of your team, where target workspace exists.
team_name = "marcwang17" # Automatically inserted

# Change this field to the of your workspace, where target project exists.
workspace_name = "First Workspace" # Automatically inserted

# Change this field to the name of your target project.
project_name = "synthetic_data" # Automatically inserted

# If you are running this notebook on a Supervisely web instance, the connection
# Edit those values if you run this notebook on your own PC
# details below will be filled in from environment variables automatically.
#
# If you are running this notebook locally on your own machine, edit to fill in the
# connection details manually. You can find your access token at
# "Your name on the top right" -> "Account settings" -> "API token".
address = os.environ['SERVER_ADDRESS']
token = os.environ['API_TOKEN']

In [3]:
# Initialize API object
api = sly.Api(address, token)

In [4]:
team = api.team.get_info_by_name(team_name)
if team is None:
    raise RuntimeError("Team {!r} not found".format(team_name))

workspace = api.workspace.get_info_by_name(team.id, workspace_name)
if workspace is None:
    raise RuntimeError("Workspace {!r} not found".format(workspace_name))
    
print("Team: id={}, name={}".format(team.id, team.name))
print("Workspace: id={}, name={}".format(workspace.id, workspace.name))

Team: id=25454, name=marcwang17
Workspace: id=28720, name=First Workspace


In [5]:
synt_project = api.project.get_info_by_name(workspace.id, project_name)

if synt_project is None:
    synt_project = api.project.create(workspace.id, project_name, change_name_if_conflict=True)
    
    particle_meta = ObjClass('particle', Bitmap, [255,0,0]) 
    
    meta = ProjectMeta.from_json(api.project.get_meta(synt_project.id))
    meta = meta.add_obj_class(particle_meta)
    api.project.update_meta(synt_project.id, meta.to_json())
else:
    meta = ProjectMeta.from_json(api.project.get_meta(synt_project.id))
    particle_meta = meta.get_obj_class('particle')

In [6]:
class cb_annotation:
    def __init__(self, ann, obj_class):
        self.labels = ann
        self.ann_type = obj_class
    
    def __call__(self, strel, r, x, y):
        elem = strel.copy()
        x_min = x-r
        y_min = y-r
        origin = PointLocation(x_min, y_min)
        mask = Bitmap(elem, origin)
        
        label_obj = Label(mask, self.ann_type)
        
        self.labels = self.labels.add_label(label_obj)

In [ ]:
def remap_phase(im, void_mean, void_std, solid_mean, solid_std):
    n_im = np.zeros_like(im, dtype = np.uint8)
    

In [ ]:
test_im = np.zeros((600,600), dtype=bool)
ann = Annotation(test_im.shape)
cb_func = cb_annotation(ann, particle_meta)    
sphere_pack = pg.RSA(np.zeros((600,600), dtype=int), 20, 0.5, callback = cb_func)
rescaled_im = exposure.rescale_intensity(sphere_pack, in_range='image', out_range=np.uint8)

ann = cb_func.labels
ann_r = ann.scale(1/6)

ds_im = transform.downscale_local_mean(rescaled_im, (6,6))

rgb_im = color.gray2rgb(ds_im)

dataset = api.dataset.create(synt_project.id, 'synt_test', change_name_if_conflict=True)
test = api.image.upload_np(dataset.id, 'IM1.png', rgb_im)

api.annotation.upload_ann(test.id, ann_r)